cones.ipynb

- creates obj representation of borehole cone (with spherical base)


In [305]:
import os
import math
from dataclasses import dataclass

In [306]:
@dataclass
class Point:
    x: float # x-coordinate
    y: float # y-coordinate
    z: float # z-coordinate

    def __post_init__(self):
        '''
        validate dataclass attributes
        '''
        for i in (self.x, self.y, self.z):
            if not (isinstance(i, float) or isinstance(i, int)):
                raise ValueError('x, y, z must be numbers (int, float)')

In [307]:
@dataclass
class Cone:
    point: Point # point on surface where drilling starts
    length: float # borehole length
    sides: int # number of sides of base polygon
    rings: int # number of 'rings' (needed to approximate sphere surface)
    dip: float # dip of borehole, measured from horizontal plane

    SIDES_MIN = 3 # minimum number of sides of base polygon
    RINGS_MIN = 1 # minimum number of 'rings'
    DIP_MIN, DIP_MAX = 0, 90 # range for borehole dip (in degrees)

    def __post_init__(self):
        '''
        validate dataclass attributes
        '''
        if not isinstance(self.point, Point):
            raise ValueError('point must be an instance of Point')
        if not self.length > 0:
            raise ValueError('length must be a positive float')
        for i in (self.sides, self.rings):
            if not isinstance(i, int):
                raise ValueError('sides and rings must be an integers')
        if self.sides < self.SIDES_MIN:
            self.sides = self.SIDES_MIN
            print(f'Number of sides set to {self.SIDES_MIN}')
        if self.rings < self.RINGS_MIN:
            self.rings = self.RINGS_MIN
            print(f'Number of rings set to {self.RINGS_MIN}')
        if self.dip <= self.DIP_MIN or self.dip >= self.DIP_MAX:
            raise ValueError(
                f'dip must be larger than ({self.DIP_MIN} and smaller than {self.DIP_MAX}]')
        self.dip = math.radians(self.dip)

    @property
    def points_polar(self):
        '''
        points for sphere approximation (in polar coordinates)
        '''
        pts = []
        for i in range(self.rings + 1):
            radius = i * self.length * math.cos(self.dip) / self.rings
            height =  - math.sqrt(self.length**2 - (i * self.length * math.cos(self.dip) / self.rings)**2)
            for j in range(self.sides): 
                angle = j * 2 * math.pi / self.sides
                if i == 0 and j > 0:
                    pass # avoid duplicate points on ring 0
                else:
                    pts.append((i, radius, angle, height))
        pts.append((-1, 0, 0, 0)) # add point on surface w/ index -1
        return pts

    @property
    def points_cartesian(self):
        '''
        points for sphere approximation (in Cartesian coordinates)
        '''
        return [[
            p[0],
            self.point.x + p[1] * math.cos(p[2]),
            self.point.y + p[1] * math.sin(p[2]),
            self.point.z + p[3]
            ] for p in self.points_polar]

    @staticmethod
    def _shift(seq, n):
        '''
        shift iterbale seq by n
        '''
        n = n % len(seq)
        return seq[n:] + seq[:n]

    @property
    def indices(self):
        '''
        enumerate points
        '''
        return [{
            'ring': i[0], 'idx': idx + 1
            } for idx, i in enumerate(self.points_cartesian)]

    @property
    def faces(self):
        '''
        create triangles and quadrilaterals from Cartesian points
        '''
        pts_idx = self.indices
        faces = []
        for i in range(self.rings):
            ring_current = [pt for pt in pts_idx if pt['ring'] == i]
            ring_next = [pt for pt in pts_idx if pt['ring'] == i + 1]
            if i == 0:
                # add triangles
                idx1 = ring_current[0]['idx']
                idx2 = [pt['idx'] for pt in ring_next]
                idx3 = self._shift(idx2, 1)
                faces.extend([(idx1, i2, i3) for i2, i3 in zip(idx2, idx3)])
            elif i > 0:
                # add quadrilaterals
                # obj only supports triangles: 1 quadrilaterals = 2 triangles
                idx1 = [pt['idx'] for pt in ring_current]
                idx2 = self._shift(idx1, 1)
                idx3 = [pt['idx'] for pt in ring_next]
                idx4 = self._shift(idx3, 1)
                faces.extend([(
                    i1, i2, i3
                    ) for i1, i2, i3 in zip(idx1, idx2, idx3)])
                faces.extend([(
                    i2, i3, i4
                    ) for i2, i3, i4 in zip(idx2, idx3, idx4)])
                # add triangles between outer ring vertices and point on surface
                if i == self.rings - 1:
                    idx1 = pts_idx[-1]['idx'] # point on surface
                    faces.extend([(idx1, i3, i4) for i3, i4 in zip(idx3, idx4)])
        return faces

    def create_obj(self, name, directory=''):
        if directory and not os.path.isdir(directory):
            raise ValueError('directory does not exist')
        points = [(
            round(p[1], 3),
            round(p[2], 3),
            round(p[3], 3)) for p in self.points_cartesian]
        faces = self.faces
        full_path = os.path.join(directory, name + '.obj')
        with open(full_path, 'w') as file:
            for p in points:
                file.write(f'v {p[0]} {p[1]} {p[2]}\n')
            for f in faces:
                f_as_strings = [str(fi) for fi in f]
                string = ' '.join(f_as_strings)
                file.write(f'f {string}\n')
        print('obj file created')
        

TESTS

In [312]:
# create cone, create obj file

point = Point(109161, 1216078, 149) # Point(x, y, z)
cone = Cone(point, 200, 200, 200, 60) # Cone(Point, length, sides, rings, dip)
cone.create_obj('test')

obj file created
